<a href="https://colab.research.google.com/github/MKrupauskas/colab/blob/master/federated-learning-aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 256kB 38.0MB/s 
     |████████████████████████████████| 389kB 38.5MB/s 
     |████████████████████████████████| 204kB 43.6MB/s 
     |████████████████████████████████| 460kB 23.1MB/s 
     |████████████████████████████████| 2.1MB 23.7MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 266kB 36.5MB/s 
     |████████████████████████████████| 51kB 18.1MB/s 
     |████████████████████████████████| 122kB 31.0MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067077 sha256=3a7864173b2e4e45b9ee433a3bf6708f2d18420b8e9a872ca3a23adc0f2706ec
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=2281cd7390b12b8e632c4ee4810a0c3ce41a98e235218fe437a21bdb9adb93ad
  Stored in dir

In [2]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0817 16:38:30.114596 140429852317568 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0817 16:38:30.136445 140429852317568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")

secure_worker = sy.VirtualWorker(hook, id = "secure_worker")

In [5]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0817 16:41:01.288561 140429852317568 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0817 16:41:01.296986 140429852317568 base.py:646] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0817 16:41:01.300031 140429852317568 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0817 16:41:01.303591 140429852317568 base.py:646] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0817 16:41:01.305036 140429852317568 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0817 16:41:01.306737 140429852317568 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
data = th.tensor([[1., 1], [0, 1,], [1, 0], [0, 0]], requires_grad = True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad = True)

In [0]:
bobs_data = data[0 : 2].send(bob)
bobs_target = target[0 : 2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
model = nn.Linear(2, 1)

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [0]:
bobs_optimizer = optim.SGD(params = bobs_model.parameters(), lr = 0.1)
alices_optimizer = optim.SGD(params = alices_model.parameters(), lr = 0.1)

In [23]:
bobs_optimizer.zero_grad()

bobs_prediction = bobs_model(bobs_data)

bobs_loss = ((bobs_prediction - bobs_target) ** 2).sum()

bobs_loss.backward()

bobs_optimizer.step()

bobs_loss = bobs_loss.get().data

bobs_loss

tensor(0.2459)

In [25]:
alices_optimizer.zero_grad()

alices_prediction = alices_model(alices_data)

alices_loss = ((alices_prediction - alices_target) ** 2).sum()

alices_loss.backward()

alices_optimizer.step()

alices_loss = alices_loss.get().data

alices_loss

tensor(0.2677)

In [0]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [32]:
model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

secure_worker.clear_objects()

RuntimeError: ignored

In [0]:
import random

Q = 23740629843760239486723

def encrypt(x = 5, n_shares = 3):
  shares = list()

  for i in range(n_shares - 1):
    shares.append(random.randint(0, Q))

  final_share = Q - (sum(shares) % Q) + x

  shares.append(final_share)

  return tuple(shares)

In [42]:
encrypt(5, 10)

(18117822702519624652697,
 16845955571467678648776,
 12929178515075397051059,
 1959862978385802812378,
 22177967570868865474481,
 16863424841757678966079,
 7678460852548205736740,
 22274349182943953886863,
 8833636086954840497348,
 14763120760039389193922)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [46]:
decrypt(encrypt())

5

In [0]:
def add(a, b):
  c = list()

  assert(len(a) == len(b))

  for i in range(len(a)):
    c.append((a[i] + b[i]) % Q)

  return tuple(c)

In [50]:
decrypt(add(encrypt(5), encrypt(2)))

7

For floating point numbers

In [55]:
BASE = 10
PRECISION = 4

def encode(x_dec):
  return int(x_dec * (BASE ** PRECISION)) % Q

def decode(x_fp):
  return (x_fp if x_fp <= Q / 2 else x_fp - Q) / BASE ** PRECISION

0.5

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [58]:
x = th.tensor([1,2,3,4,5])

x = x.share(bob, alice, secure_worker)

y = x + x

y.get()

tensor([ 2,  4,  6,  8, 10])

In [59]:
x = th.tensor([0.1,0.2,0.3,0.4,0.5])

x = x.fix_prec()

x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])